In [1]:
import imp
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torchvision
from torch.utils.data import DataLoader, Dataset
from opacus import PrivacyEngine
import torch.nn.functional as F
from operations import *
from genotypes import PRIMITIVES
from genotypes import Genotype
from opacus.grad_sample import GradSampleModule, register_grad_sampler
from opacus.utils.batch_memory_manager import BatchMemoryManager
from typing import Dict
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from copy import deepcopy

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# Finding sample-wise Gradients for Supernet

In this notebook we derive how we come up with sample-wise gradients for our supernet we use to perform differentiable NAS. For this we note that the supernet is based on a convex, weighted combination of operations which are all applied to the same input, that is each _mixed operation_ is defined as follows:
\begin{equation}
    m = \sum_{o \in O} \alpha_o \cdot o(x)
\end{equation}
Each $o$ is a convolution/pooling operation or a regular neural network, thus opacus already knows how to compute sample-based gradients for all parameters of each $o$. Thus, with a smart design of our supernet-architecture we can avoid the computation of sample-wise gradients for all the operations we have in use and pass the heavy lifting to opacus. The problem then reduces to providing opacus with sample-wise gradients w.r.t $\alpha_o$ for each $o$.

For this, let's see how we compute the gradients of an arbitrary loss w.r.t. the alpha-parameters in a simple setup: We only have 3 operations, each associated with a certaing weight $\alpha_o$. The mixed operation is then followed by a linear transformation producing the output, thus the network reads:
\begin{equation}
    \hat{y} = \bigg(\sum_{o \in O} \alpha_o \cdot o(x) \bigg) \cdot \mathbf{W}
\end{equation}

The following shwos the forward pass of such a network:

In [194]:
X = [torch.randn(1, 5) for _ in range(0, 4)]
alphas = nn.Parameter(torch.ones(4) / 4, requires_grad=True)
W = nn.Parameter(torch.randn(5, 1), requires_grad=True)

In [195]:
softmaxed_alphas = torch.softmax(alphas, dim=0)
softmaxed_alphas

tensor([0.2500, 0.2500, 0.2500, 0.2500], grad_fn=<SoftmaxBackward0>)

In [196]:
mop = sum(alphas[i] * X[i] for i in range(0, 4))
y = torch.matmul(mop, W)
y

tensor([[1.1655]], grad_fn=<MmBackward0>)

Let's proceed with computing a loss and updating the parameters. To keep things easy, let's pretend our loss is just the sum of all elements in our output. We then can compute the gradients w.r.t. each $\alpha_o$ easily by calling the backward-method:

In [197]:
l = torch.sum(y)
l.backward()
alphas.grad

tensor([-1.5030,  3.3426, -0.0513,  2.8738])

The gradient can be expressed as follows:
\begin{equation}
    \frac{\partial \ell}{\partial \alpha_o} = \sum_{i=1}^n \frac{\partial \ell}{\partial z^{(0)}_k} \cdot \frac{\partial z^{(0)}_k}{\partial \alpha_o}
\end{equation}
Here we sum over all gradients of our $n$ outputs w.r.t. $\alpha_o$. $z^{(0)}_k$ denotes the $k$-th element of the last layer (output). Expanding this further yields:
\begin{align}
    \frac{\partial \ell}{\partial \alpha_o} = & \sum_{i=1}^n \frac{\partial \ell}{\partial z^{(0)}_k} \cdot \sum_{d=1}^{|L_1|} \mathbf{W}^{(1)}_{d k} \cdot \frac{\partial z_d^{(1)}}{\partial \alpha_o} \\
    & \sum_{i=1}^n \frac{\partial \ell}{\partial z^{(0)}_k} \cdot \sum_{d=1}^{|L_1|} \mathbf{W}^{(1)}_{d k} \cdot \frac{\partial \sum_{o \in O} \big( \alpha_o o(z^{(2)}) \big)}{\partial \alpha_o} \\
    &  \sum_{i=1}^n \frac{\partial \ell}{\partial z^{(0)}_k} \cdot \sum_{d=1}^{|L_1|} \mathbf{W}^{(1)}_{d k} \cdot o(z^{(2)})_d
\end{align}
Since $\frac{\partial \ell}{\partial z^{(0)}_k} = 1$ we obtain:
\begin{align}
    \frac{\partial \ell}{\partial \alpha_o} & = \sum_{k=1}^n \sum_{d=1}^{|L_1|} \mathbf{W}_{d k}^{(1)} \cdot o(z^{(2)})_d \\
\end{align}
This reduces to:
\begin{equation}
    \frac{\partial \ell}{\partial \alpha_o} = \sum_{k=1}^n \big(\mathbf{W}^{(1)^T}\big)_k \cdot o(z^{(2)})
\end{equation}

In [198]:
X = torch.cat(X, dim=0)
1 * X.matmul(W) # 1 = derivative w.r.t. the output of mixed operation, rest as derived above

tensor([[-1.5030],
        [ 3.3426],
        [-0.0513],
        [ 2.8738]], grad_fn=<MulBackward0>)

## Extending the Minimal Example
Now we will start extending the above approach. As you can see computing the gradients and deriving the forumlas can get cumbersome really quickly. That's why we should use opacus' capabilities of computing sample-wise gradients for as many modules as possible. Below we define 3 operations that will be used withing our NAS-approach. For these modules opacus already knows how to compute sample-wise gradients, thus we can just go ahead and use them as they are.

In [2]:
class Op1(nn.Module):

    def __init__(self, in_dim, h_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x

class Op2(nn.Module):

    def __init__(self, in_dim, h_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

class Op3(nn.Module):

    def __init__(self, in_dim, h_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.fc2 = nn.Linear(h_dim, out_dim)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        return x

PRIMS = [Op1, Op2, Op3]

The tricky part is the mixed operation-module. If we would use plain pytorch, we just could go ahead and build one `MixedOp` module taking care of calling each operation and compute the convex combination of the operation's output. However, since opacus does not know the MixedOp-module it cannot compute the sample-wise gradients w.r.t. the alpha-parameters. Thus we have to tell opacus how to compute the gradients. For this opacus provides the activations (input to our module) and the gradients w.r.t. the outputs of our module. If we would go for a "plain-pytorch approach" this would require us to compute the gradients w.r.t. alpha-parameters and the model-parameters of each operation "by hand". Since this is cumbersome and not error-prone, we split up the MixedOp into two parts: One `ParallelOp` which does nothing but applying each operation on the same input data and a `MixedOp` (please don't get confused by the naming) which just cares about computing the convex combination of the oerpation's output computed by the ParallelOp. 

This way we can compute the gradients w.r.t. the alphas easily by just applying the same reasoning as above. The gradients can then be computed by computing the vector-product of the activations and the gradients w.r.t. the MixedOp (which are both provided by opacus). Mathematically this can be expressed as follows assuming we have an $n \times i$-dimensional real activation matrix $\mathbf{a}$ and an $i$-dimensional real vector $\nabla \mathbf{m}$ representing the gradients w.r.t. our mixed operation. We have an activation of $n \times i$ because we compute the convex combination of $n$ operations, each producing outputs of dimension $i$. Since each of the $n$ operations is associated with a weight $\alpha_j$, we aim to compute the gradient w.r.t. each of the $n$ weights, thus we aim to obtain a $n$-dimensional gradient vector for one sample and a $B \times n$-dimensional gradient matrix for a batch of size $B$. We can easily compute this using an einsum:
\begin{equation}
    \nabla_{j} \alpha_m = \sum_{k=1}^i \nabla \mathbf{m}_i \cdot \mathbf{a}_{m k}
\end{equation}
Here $\nabla_j \alpha_m$ is the $j$-th element of the gradient vector w.r.t. to the alphas associated with operation $m$. As we can see this is just a more general version of the equation we've derived above.

But this is not all we have to do: Remember we have the ParallelOp computing all the operation's outputs. This module does not have any parameters, thus there is nothing we can update and so there are also no gradients for this module. Thus we can tell opacus that there is nothing to compute in the backward pass. Details on the implementation can be obtained below:

In [2]:
class ParallelOp(nn.Module):
    
    def __init__(self, in_dim, h_dim, out_dim) -> None:
        super(ParallelOp, self).__init__()
        self._ops = nn.ModuleList()
        for primitive in PRIMS:
            self._ops.append(primitive(in_dim, h_dim, out_dim))

    def forward(self, x):
        operation_outs = []
        for op in self._ops:
            out = op(x)
            operation_outs.append(out)
        return torch.stack(operation_outs)

In [3]:
class MixedOp(nn.Module):

    def __init__(self):
        super(MixedOp, self).__init__()
        self.alphas = nn.Parameter(torch.zeros(len(PRIMS)), requires_grad=True)

    def forward(self, x):
        weights = torch.softmax(self.alphas, 0)
        return sum(w * op_out for w, op_out in zip(weights, x))

    def arch_params(self):
        return [self.alphas]

In [4]:
class LastLayer(nn.Module):

    def __init__(self, in_dim, hdim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_dim, hdim)
        self.fc2 = nn.Linear(hdim, out_dim)
        
    def forward(self, x):
        x = torch.hstack(x)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [6]:
mixed_op = MixedOp()
net = nn.Sequential(ParallelOp(28*28, 512, 256), mixed_op, LastLayer())
optim = torch.optim.SGD(mixed_op.arch_params(), 0.01)
loss = torch.nn.CrossEntropyLoss()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0,), (1,))])
train_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=True, transform=transform)
val_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=False, transform=transform)
train_loader = DataLoader(train_data, 64)
val_loader = DataLoader(val_data, 64)

@register_grad_sampler(ParallelOp)
def grad_sampler_parallel_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    return {}

@register_grad_sampler(MixedOp)
def grad_sampler_mixed_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    grad = torch.einsum('nbi,bi->nb', activations, backprops)
    ret = {
        layer.alphas: grad
    }
    return ret

pe = PrivacyEngine()
netc = deepcopy(net)
net_, optim_, train_loader_ = pe.make_private(module=net, optimizer=optim, data_loader=train_loader, noise_multiplier=1., max_grad_norm=1.)
x_first, y_first = next(iter(train_loader_))
x_first = x_first.reshape((x_first.shape[0], 28*28))

TypeError: __init__() missing 3 required positional arguments: 'in_dim', 'hdim', and 'out_dim'

In [ ]:
x_first = torch.randn((64, 784))
y_first = torch.ones(64, dtype=torch.long)

In [12]:
for pnetc, pnet_ in zip(netc.parameters(), net_.parameters()):
    print(torch.all(pnet_.data == pnetc.data))

NameError: name 'netc' is not defined

In [46]:
y_pred = net_(x_first)
l = loss(y_pred, y_first)
l.backward()

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [47]:
y_pred = netc(x_first)
l = loss(y_pred, y_first)
l.backward()

In [48]:
for pnetc, pnet_ in zip(netc.parameters(), net_.parameters()):
    print(torch.all(pnetc.grad.data == pnet_.grad.data))

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


> NOTE: Does this make sense? 
> 
> Yes it does since the optimizer performs the clipping and adds noise, thus the gradients are the same

## Building up a Cell
Now that we know how to build MixedOps with minimal effort, we can use the MixedOps in order to build up cells and we will use the cells in turn to build up our architecture search space. Since cells don't introduce any additional parameters, building a cell based on a set of MixedOps shoud be straightforward.

In [5]:
class Cell(nn.Module):
    # TODO: Do the same as above and check that gradients are correct!
    def __init__(self) -> None:
        super().__init__()
        self.nodes = nn.ModuleList()
        # initialize stem modules with some conv-operation self.stem0, self.stem1 = 
        curr_in_dim = 28*28
        curr_hdim = int(0.75 * curr_in_dim)
        curr_out_dim = int(0.75 * curr_hdim)
        dims = [curr_out_dim]
        for i in range(5):
            if i == 0:
                mop = nn.Sequential(ParallelOp(curr_in_dim, curr_hdim, curr_out_dim), MixedOp())
            else:
                in_dim = sum(dims)
                hdim = int(0.75*in_dim)
                out_dim = int(0.75*hdim)
                dims.append(out_dim)
                mop = nn.Sequential(ParallelOp(in_dim, hdim, out_dim), MixedOp())
            self.nodes.append(mop)

        
    def forward(self, x):
        inp = [self.nodes[0](x)]
        for op in self.nodes[1:]:
            print(torch.hstack(inp).shape)
            out = op(torch.hstack(inp))
            inp.append(out)
        
        return inp

## Building a Supernet
Now that we know how to build up a cell we can proceed and glue several cells to one network.

In [6]:
class Network(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.cell1 = Cell()
        self.linear1 = nn.Linear(2623, 28*28)
        self.cell2 = Cell()
        self.out = LastLayer(2623, 256, 10)

    def forward(self, x):
        x = self.cell1(x)
        x = torch.hstack(x)
        x = self.linear1(x)
        x = self.cell2(x)
        return self.out(x)

In [7]:
def get_params(net: nn.Module, param_type='arch'):
    parameters = []
    for name, param in net.named_parameters():
        if param_type == 'arch':
            if 'alphas' in name:
                parameters.append(param)
        elif param_type == 'model':
            if 'alphas' not in name:
                parameters.append(param)
        else:
            raise ValueError('Unsupported parameter type, must be either arch or model')
    return parameters

In [9]:
#net = nn.Sequential(Cell(), LastLayer(2623, 256, 10))
net = Network()
params = get_params(net, 'arch')
optim = torch.optim.SGD(params, 0.01)
loss = torch.nn.CrossEntropyLoss()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0,), (1,))])
train_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=True, transform=transform)
val_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=False, transform=transform)
train_loader = DataLoader(train_data, 64)
val_loader = DataLoader(val_data, 64)

@register_grad_sampler(ParallelOp)
def grad_sampler_parallel_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    return {}

@register_grad_sampler(MixedOp)
def grad_sampler_mixed_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    grad = torch.einsum('nbi,bi->nb', activations, backprops)
    ret = {
        layer.alphas: grad
    }
    return ret

pe = PrivacyEngine()
netc = deepcopy(net)
net_, optim_, train_loader_ = pe.make_private(module=net, optimizer=optim, data_loader=train_loader, noise_multiplier=1., max_grad_norm=1.)
x_first, y_first = next(iter(train_loader_))
x_first = x_first.reshape((x_first.shape[0], 28*28))

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/opacus/privacy_engine.py:130: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [15]:
x_first = torch.randn((64, 784))
y_first = torch.ones(64, dtype=torch.long)

In [18]:
y_pred = net_(x_first)
l = loss(y_pred, y_first)
l.backward()

y_pred = netc(x_first)
l = loss(y_pred, y_first)
l.backward()

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


torch.Size([64, 441])
torch.Size([64, 688])
torch.Size([64, 1075])
torch.Size([64, 1679])
torch.Size([64, 441])
torch.Size([64, 688])
torch.Size([64, 1075])
torch.Size([64, 1679])
torch.Size([64, 441])
torch.Size([64, 688])
torch.Size([64, 1075])
torch.Size([64, 1679])
torch.Size([64, 441])
torch.Size([64, 688])
torch.Size([64, 1075])
torch.Size([64, 1679])


In [19]:
for pnetc, pnet_ in zip(netc.parameters(), net_.parameters()):
    print(torch.all(pnetc.grad.data == pnet_.grad.data))

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

# Transfer to CNNs
In the above example we have used standard MLPs as our operations. However, our ultimate goal is to perform image classification, thus our operation-space consists of convolution- and pooling-operations. This requires us to adapt the computation of the gradients slightly.

In [8]:
class MixedOp(nn.Module):

  def __init__(self, C, stride):
    super(MixedOp, self).__init__()
    self._ops = nn.ModuleList()
    for primitive in PRIMITIVES:
      op = OPS[primitive](C, stride, False)
      if 'pool' in primitive:
        op = nn.Sequential(op, nn.GroupNorm(num_groups=1, num_channels=C, affine=False))
      self._ops.append(op)

  def forward(self, x, weights):
    return sum(w * op(x) for w, op in zip(weights, self._ops))


class Cell(nn.Module):

  def __init__(self, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev):
    super(Cell, self).__init__()
    self.reduction = reduction

    if reduction_prev:
      self.preprocess0 = FactorizedReduce(C_prev_prev, C, affine=False)
    else:
      self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 0, affine=False)
    self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 0, affine=False)
    self._steps = steps
    self._multiplier = multiplier

    self._ops = nn.ModuleList()
    self._bns = nn.ModuleList()
    for i in range(self._steps):
      for j in range(2+i):
        stride = 2 if reduction and j < 2 else 1
        op = MixedOp(C, stride)
        self._ops.append(op)

  def forward(self, s0, s1, weights):
    s0 = self.preprocess0(s0)
    s1 = self.preprocess1(s1)

    states = [s0, s1]
    offset = 0
    for i in range(self._steps):
      s = sum(self._ops[offset+j](h, weights[offset+j]) for j, h in enumerate(states))
      offset += len(states)
      states.append(s)

    return torch.cat(states[-self._multiplier:], dim=1)


class Network(nn.Module):

  def __init__(self, C, num_classes, layers, criterion, device, in_channels=3, steps=4, multiplier=4, stem_multiplier=3):
    super(Network, self).__init__()
    self._C = C
    self._num_classes = num_classes
    self._layers = layers
    self._criterion = criterion
    self._steps = steps
    self._multiplier = multiplier
    self.device = device

    C_curr = stem_multiplier*C
    self.stem = nn.Sequential(
      nn.Conv2d(in_channels, C_curr, 3, padding=1, bias=False),
      nn.GroupNorm(num_groups=1, num_channels=C_curr),
    )
 
    C_prev_prev, C_prev, C_curr = C_curr, C_curr, C
    self.cells = nn.ModuleList()
    reduction_prev = False
    for i in range(layers):
      if i in [layers//3, 2*layers//3]:
        C_curr *= 2
        reduction = True
      else:
        reduction = False
      cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr, reduction, reduction_prev)
      reduction_prev = reduction
      self.cells += [cell]
      C_prev_prev, C_prev = C_prev, multiplier*C_curr

    self.global_pooling = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(C_prev, num_classes)

    self._initialize_alphas()

  def new(self):
    model_new = Network(self._C, self._num_classes, self._layers, self._criterion, self.device).to(self.device)
    for x, y in zip(model_new.arch_parameters(), self.arch_parameters()):
        x.data.copy_(y.data)
    return model_new

  def forward(self, input):
    s0 = s1 = self.stem(input)
    for i, cell in enumerate(self.cells):
      if cell.reduction:
        weights = F.softmax(self.alphas_reduce, dim=-1)
      else:
        weights = F.softmax(self.alphas_normal, dim=-1)
      s0, s1 = s1, cell(s0, s1, weights)
    out = self.global_pooling(s1)
    logits = self.classifier(out.view(out.size(0),-1))
    return logits

  def _initialize_alphas(self):
    k = sum(1 for i in range(self._steps) for n in range(2+i))
    num_ops = len(PRIMITIVES)

    self.alphas_normal = nn.Parameter(1e-3*torch.zeros((k, num_ops)).to(self.device), requires_grad=True)
    self.alphas_reduce = nn.Parameter(1e-3*torch.zeros((k, num_ops)).to(self.device), requires_grad=True)
    self._arch_parameters = [
      self.alphas_normal,
      self.alphas_reduce,
    ]

  def arch_parameters(self):
    return self._arch_parameters

  def genotype(self):

    def _parse(weights):
      gene = []
      n = 2
      start = 0
      for i in range(self._steps):
        end = start + n
        W = weights[start:end].copy()
        edges = sorted(range(i + 2), key=lambda x: -max(W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
        for j in edges:
          k_best = None
          for k in range(len(W[j])):
            if k != PRIMITIVES.index('none'):
              if k_best is None or W[j][k] > W[j][k_best]:
                k_best = k
          gene.append((PRIMITIVES[k_best], j))
        start = end
        n += 1
      return gene

    gene_normal = _parse(F.softmax(self.alphas_normal, dim=-1).data.cpu().numpy())
    gene_reduce = _parse(F.softmax(self.alphas_reduce, dim=-1).data.cpu().numpy())

    concat = range(2+self._steps-self._multiplier, self._steps+2)
    genotype = Genotype(
      normal=gene_normal, normal_concat=concat,
      reduce=gene_reduce, reduce_concat=concat
    )
    return genotype

In [9]:
criterion = nn.CrossEntropyLoss()
device = torch.device('cpu')
model = Network(16, 10, 4, criterion, device, in_channels=1) # Cell(4, 3, 16, 36, 48, False, False)
optim_arch = torch.optim.SGD(get_params(model, 'arch'), 0.01)
optim_model = torch.optim.SGD(get_params(model, 'model'), 0.01)
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0,), (1,))])
train_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=True, transform=transform)
val_data = torchvision.datasets.FashionMNIST('../../datasets/femnist/', download=True, train=False, transform=transform)
train_loader = DataLoader(train_data, 64)
val_loader = DataLoader(val_data, 64)

In [10]:
# initialize all weights to zero
for param in get_params(model, 'model'):
    param.data.fill_(0.1)

for param in get_params(model, 'model'):
    print(torch.all(param.data == 0.1))

for param in get_params(model, 'arch'):
    print(torch.all(param.data == 0))

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

In [11]:
class ParallelOp(nn.Module):

  def __init__(self, C, stride) -> None:
    super().__init__()
    self._ops = nn.ModuleList()
    for primitive in PRIMITIVES:
      op = OPS[primitive](C, stride, False)
      if 'pool' in primitive:
        op = nn.Sequential(op, nn.GroupNorm(num_groups=1, num_channels=C, affine=False))
      self._ops.append(op)

  def forward(self, x):
      operation_outs = []
      for op in self._ops:
          out = op(x)
          operation_outs.append(out)
      return torch.stack(operation_outs)

class MixedOp(nn.Module):

    def __init__(self):
        super(MixedOp, self).__init__()
        self.alphas = nn.Parameter(torch.zeros(len(PRIMITIVES)), requires_grad=True)

    def forward(self, x):
      weights = torch.softmax(self.alphas, 0)
      return sum(w * op_out for w, op_out in zip(weights, x))


class Cell(nn.Module):

  def __init__(self, steps, multiplier, C_prev_prev, C_prev, C, reduction, reduction_prev, mixed_ops_normal, mixed_ops_reduce):
    super(Cell, self).__init__()
    self.reduction = reduction

    if reduction_prev:
      self.preprocess0 = FactorizedReduce(C_prev_prev, C, affine=False)
    else:
      self.preprocess0 = ReLUConvBN(C_prev_prev, C, 1, 1, 0, affine=False)
    self.preprocess1 = ReLUConvBN(C_prev, C, 1, 1, 0, affine=False)
    self._steps = steps
    self._multiplier = multiplier

    self._ops = nn.ModuleList()
    self._bns = nn.ModuleList()

    mixed_op_idx = 0
    for i in range(self._steps):
      for j in range(2+i):
        stride = 2 if reduction and j < 2 else 1
        if reduction:
          op = nn.Sequential(ParallelOp(C, stride), mixed_ops_reduce[mixed_op_idx])
        else:
          op = nn.Sequential(ParallelOp(C, stride), mixed_ops_normal[mixed_op_idx])
        mixed_op_idx += 1
        self._ops.append(op)

  def forward(self, s0, s1):
    s0 = self.preprocess0(s0)
    s1 = self.preprocess1(s1)

    states = [s0, s1]
    offset = 0
    for i in range(self._steps):
      s = sum(self._ops[offset+j](h) for j, h in enumerate(states))
      offset += len(states)
      states.append(s)

    return torch.cat(states[-self._multiplier:], dim=1)


class Network(nn.Module):

  def __init__(self, C, num_classes, layers, criterion, device, in_channels=3, steps=4, multiplier=4, stem_multiplier=3):
    super(Network, self).__init__()
    self._C = C
    self._num_classes = num_classes
    self._layers = layers
    self._criterion = criterion
    self._steps = steps
    self._multiplier = multiplier
    self.device = device

    C_curr = stem_multiplier*C
    self.stem = nn.Sequential(
      nn.Conv2d(in_channels, C_curr, 3, padding=1, bias=False),
      nn.GroupNorm(num_groups=1, num_channels=C_curr),
    )
 
    C_prev_prev, C_prev, C_curr = C_curr, C_curr, C
    self.cells = nn.ModuleList()
    reduction_prev = False
    self._init_mixed_ops()
    for i in range(layers):
      if i in [layers//3, 2*layers//3]:
        C_curr *= 2
        reduction = True
      else:
        reduction = False

      cell = Cell(steps, multiplier, C_prev_prev, C_prev, C_curr, reduction, reduction_prev, self.mixed_ops_normal, self.mixed_ops_reduce)
      reduction_prev = reduction
      self.cells += [cell]
      C_prev_prev, C_prev = C_prev, multiplier*C_curr

    self.global_pooling = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(C_prev, num_classes)

  def new(self):
    model_new = Network(self._C, self._num_classes, self._layers, self._criterion, self.device).to(self.device)
    for x, y in zip(get_params(model_new, 'arch'), get_params(self, 'arch')):
        x.data.copy_(y.data)
    return model_new

  def forward(self, input):
    s0 = s1 = self.stem(input)
    for i, cell in enumerate(self.cells):
      s0, s1 = s1, cell(s0, s1)
    out = self.global_pooling(s1)
    logits = self.classifier(out.view(out.size(0),-1))
    return logits

  def _init_mixed_ops(self):
    k = sum(1 for i in range(self._steps) for n in range(2+i))
    self.mixed_ops_normal = [MixedOp() for _ in range(k)]
    self.mixed_ops_reduce = [MixedOp() for _ in range(k)]

  def genotype(self):

    def _parse(weights):
      gene = []
      n = 2
      start = 0
      for i in range(self._steps):
        end = start + n
        W = weights[start:end].copy()
        edges = sorted(range(i + 2), key=lambda x: -max(W[x][k] for k in range(len(W[x])) if k != PRIMITIVES.index('none')))[:2]
        for j in edges:
          k_best = None
          for k in range(len(W[j])):
            if k != PRIMITIVES.index('none'):
              if k_best is None or W[j][k] > W[j][k_best]:
                k_best = k
          gene.append((PRIMITIVES[k_best], j))
        start = end
        n += 1
      return gene

    alphas_normal = torch.stack([mop.alphas.data for mop in self.mixed_ops_normal])
    alphas_reduce = torch.stack([mop.alphas.data for mop in self.mixed_ops_reduce])
    gene_normal = _parse(F.softmax(alphas_normal, dim=-1).data.cpu().numpy())
    gene_reduce = _parse(F.softmax(alphas_reduce, dim=-1).data.cpu().numpy())

    concat = range(2+self._steps-self._multiplier, self._steps+2)
    genotype = Genotype(
      normal=gene_normal, normal_concat=concat,
      reduce=gene_reduce, reduce_concat=concat
    )
    return genotype

In [12]:
def get_flat_grad_sample(p: torch.Tensor):
    if p.grad_sample is None:
        raise ValueError(
            "Per sample gradient is not initialized. Not updated in backward pass?"
        )
    if isinstance(p.grad_sample, torch.Tensor):
        ret = p.grad_sample
    elif isinstance(p.grad_sample, list):
        ret = torch.cat(p.grad_sample, dim=0)
    else:
        raise ValueError(f"Unexpected grad_sample type: {type(p.grad_sample)}")
    return ret

In [13]:
@register_grad_sampler(ParallelOp)
def grad_sampler_parallel_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    return {}

@register_grad_sampler(MixedOp)
def grad_sampler_mixed_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    sftmx = torch.softmax(layer.alphas, 0)
    j_sftmx_vals = []
    for i in range(len(sftmx)):
        col = []
        for j in range(len(sftmx)):
            if i == j:
                deriv = sftmx[i] * (1 - sftmx[i])
            else:
                deriv = -sftmx[j] * sftmx[i]
            col.append(deriv)
        j_sftmx_vals.append(col)
    j_sftmx_trans = torch.tensor(j_sftmx_vals)
    
    #print(torch.all(activations[0] == 0))
    # d = c = number of operations, b = number of batches
    sftmx_grad = torch.einsum('dc,cb...->db...', j_sftmx_trans, activations) # we sum over columns since we have the transposed jacobian of softmax w.r.t. inputs
    final_grad = torch.einsum('db...,b...->db', sftmx_grad, backprops)
    #grad = torch.einsum('nbcwh,bcwh->nb', activations, backprops)
    ret = {
        layer.alphas: final_grad
    }
    return ret


criterion = nn.CrossEntropyLoss()
device = torch.device('cpu')
model_dp = Network(16, 10, 4, criterion, device, in_channels=1) # Cell(4, 3, 16, 36, 48, False, False)
optim_arch = torch.optim.SGD(get_params(model_dp, 'arch'), 0.01)
optim_model = torch.optim.SGD(get_params(model_dp, 'model'), 0.01)
pe = PrivacyEngine()
train_loader_c = deepcopy(train_loader)
model_dp_, optim_, train_loader_ = pe.make_private(module=model_dp, optimizer=optim_arch, data_loader=train_loader_c, noise_multiplier=1., max_grad_norm=1.)
x_first, y_first = next(iter(train_loader_))

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/opacus/privacy_engine.py:130: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [14]:
# initialize all weights to zero
for param in get_params(model_dp_, 'model'):
    param.data.fill_(0.1)

for param in get_params(model_dp_, 'model'):
    print(torch.all(param.data == 0.1))

for param in get_params(model_dp_, 'arch'):
    print(torch.all(param.data == 0))

tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)

In [213]:
y_pred1= model(x_first)
l = criterion(y_pred1, y_first)
l.backward()

y_pred2 = model_dp_(x_first)
l = criterion(y_pred2, y_first)
l.backward()

print(torch.all(y_pred1 == y_pred2))

tensor(True)


In [214]:
params_normal_dp = []
params_reduce_dp = []
alpahs_dp_model = [param for name, param in model_dp_.named_parameters() if 'alpha' in name]
for i, param in enumerate(alpahs_dp_model):
    # the first 14 entries refer to normal cell's alphas
    if i < 14:
        params_normal_dp.append(param)
    else:
        params_reduce_dp.append(param)

dp_arch_normal_params_grads = [param.grad.data for param in params_normal_dp]
dp_arch_reduce_params_grads = [param.grad.data for param in params_reduce_dp]
dp_arch_normal_params_grads = torch.stack(dp_arch_normal_params_grads)
dp_arch_reduce_params_grads = torch.stack(dp_arch_reduce_params_grads)

In [215]:
params_normal_orig = []
params_reduce_orig = []
alpahs_dp_model = [param for name, param in model_dp_.named_parameters() if 'alpha' in name]
for i, param in enumerate(alpahs_dp_model):
    # the first 14 entries refer to normal cell's alphas
    if i < 14:
        params_normal_orig.append(param)
    else:
        params_reduce_orig.append(param)

orig_arch_normal_params_grads = [param.grad.data for param in params_normal_orig]
orig_arch_reduce_params_grads = [param.grad.data for param in params_reduce_orig]
orig_arch_normal_params_grads = torch.stack(orig_arch_normal_params_grads)
orig_arch_reduce_params_grads = torch.stack(orig_arch_reduce_params_grads)

In [216]:
torch.all(orig_arch_normal_params_grads == dp_arch_normal_params_grads)

tensor(True)

In [217]:
torch.all(orig_arch_reduce_params_grads == dp_arch_reduce_params_grads)

tensor(True)

In [218]:
model_params = [param.grad.data for name, param in model.named_parameters() if 'alpha' not in name]
model_dp_params = [param.grad.data for name, param in model_dp_.named_parameters() if 'alpha' not in name]

In [219]:
# since the naming of the parameters is not aligned in the original model and the dp-model, 
# we take the classifier gradients as proxy to check if the model's gradients are the same in both models
classifier_weights, classifier_bias = None, None
for name, param in model.named_parameters():
    if name == 'classifier.weight':
        classifier_weights = param
    elif name == 'classifier.bias':
        classifier_bias = param
    else:
        pass

equals = []
for name, param in model_dp_.named_parameters():
    if name == '_module.classifier.weight':
        is_equal = torch.all(classifier_weights.grad.data == param.grad.data)
        equals.append(is_equal.item())
    elif name == '.module.classifier.bias':
        is_equal = torch.all(classifier_bias.grad.data == param.grad.data)
        equals.append(is_equal.item())
    else:
        pass

print(all(equals))

True


## Fixing optimizer-step issue

In [56]:
from typing import Union, List, Dict, Optional, Callable
from opacus.optimizers.utils import params

def _mark_as_processed(obj: Union[torch.Tensor, List[torch.Tensor]]):
    """
    Marks parameters that have already been used in the optimizer step.
    DP-SGD puts certain restrictions on how gradients can be accumulated. In particular,
    no gradient can be used twice - client must call .zero_grad() between
    optimizer steps, otherwise privacy guarantees are compromised.
    This method marks tensors that have already been used in optimizer steps to then
    check if zero_grad has been duly called.
    Notes:
          This is used to only mark ``p.grad_sample`` and ``p.summed_grad``
    Args:
        obj: tensor or a list of tensors to be marked
    """

    if isinstance(obj, torch.Tensor):
        obj._processed = True
    elif isinstance(obj, list):
        for x in obj:
            x._processed = True


def _check_processed_flag_tensor(x: torch.Tensor):
    """
    Checks if this gradient tensor has been previously used in optimization step.
    See Also:
        :meth:`~opacus.optimizers.optimizer._mark_as_processed`
    Args:
        x: gradient tensor
    Raises:
        ValueError
            If tensor has attribute ``._processed`` previously set by
            ``_mark_as_processed`` method
    """

    if hasattr(x, "_processed"):
        raise ValueError(
            "Gradients haven't been cleared since the last optimizer step. "
            "In order to obtain privacy guarantees you must call optimizer.zero_grad()"
            "on each step"
        )


def _check_processed_flag(obj: Union[torch.Tensor, List[torch.Tensor]]):
    """
    Checks if this gradient tensor (or a list of tensors) has been previously
    used in optimization step.
    See Also:
        :meth:`~opacus.optimizers.optimizer._mark_as_processed`
    Args:
        x: gradient tensor or a list of tensors
    Raises:
        ValueError
            If tensor (or at least one tensor from the list) has attribute
            ``._processed`` previously set by ``_mark_as_processed`` method
    """

    if isinstance(obj, torch.Tensor):
        _check_processed_flag_tensor(obj)
    elif isinstance(obj, list):
        for x in obj:
            _check_processed_flag_tensor(x)


def _generate_noise(
    std: float,
    reference: torch.Tensor,
    generator=None,
    secure_mode: bool = False,
) -> torch.Tensor:
    """
    Generates noise according to a Gaussian distribution with mean 0
    Args:
        std: Standard deviation of the noise
        reference: The reference Tensor to get the appropriate shape and device
            for generating the noise
        generator: The PyTorch noise generator
        secure_mode: boolean showing if "secure" noise need to be generated
            (see the notes)
    Notes:
        If `secure_mode` is enabled, the generated noise is also secure
        against the floating point representation attacks, such as the ones
        in https://arxiv.org/abs/2107.10138 and https://arxiv.org/abs/2112.05307.
        The attack for Opacus first appeared in https://arxiv.org/abs/2112.05307.
        The implemented fix is based on https://arxiv.org/abs/2107.10138 and is
        achieved through calling the Gaussian noise function 2*n times, when n=2
        (see section 5.1 in https://arxiv.org/abs/2107.10138).
        Reason for choosing n=2: n can be any number > 1. The bigger, the more
        computation needs to be done (`2n` Gaussian samples will be generated).
        The reason we chose `n=2` is that, `n=1` could be easy to break and `n>2`
        is not really necessary. The complexity of the attack is `2^p(2n-1)`.
        In PyTorch, `p=53` and so complexity is `2^53(2n-1)`. With `n=1`, we get
        `2^53` (easy to break) but with `n=2`, we get `2^159`, which is hard
        enough for an attacker to break.
    """
    zeros = torch.zeros(reference.shape, device=reference.device)
    if std == 0:
        return zeros
    # TODO: handle device transfers: generator and reference tensor
    # could be on different devices
    if secure_mode:
        torch.normal(
            mean=0,
            std=std,
            size=(1, 1),
            device=reference.device,
            generator=generator,
        )  # generate, but throw away first generated Gaussian sample
        sum = zeros
        for _ in range(4):
            sum += torch.normal(
                mean=0,
                std=std,
                size=reference.shape,
                device=reference.device,
                generator=generator,
            )
        return sum / 2
    else:
        return torch.normal(
            mean=0,
            std=std,
            size=reference.shape,
            device=reference.device,
            generator=generator,
        )

class DPOptimizer(torch.optim.Optimizer):
    """
    ``torch.optim.Optimizer`` wrapper that adds additional functionality to clip per
    sample gradients and add Gaussian noise.
    Can be used with any ``torch.optim.Optimizer`` subclass as an underlying optimizer.
    ``DPOptimzer`` assumes that parameters over which it performs optimization belong
    to GradSampleModule and therefore have the ``grad_sample`` attribute.
    On a high level ``DPOptimizer``'s step looks like this:
    1) Aggregate ``p.grad_sample`` over all parameters to calculate per sample norms
    2) Clip ``p.grad_sample`` so that per sample norm is not above threshold
    3) Aggregate clipped per sample gradients into ``p.grad``
    4) Add Gaussian noise to ``p.grad`` calibrated to a given noise multiplier and
    max grad norm limit (``std = noise_multiplier * max_grad_norm``).
    5) Call underlying optimizer to perform optimization step
    Examples:
        >>> module = MyCustomModel()
        >>> optimizer = torch.optim.SGD(module.parameters(), lr=0.1)
        >>> dp_optimizer = DPOptimizer(
        ...     optimizer=optimizer,
        ...     noise_multiplier=1.0,
        ...     max_grad_norm=1.0,
        ...     expected_batch_size=4,
        ... )
    """

    def __init__(
        self,
        optimizer: torch.optim.Optimizer,
        *,
        noise_multiplier: float,
        max_grad_norm: float,
        expected_batch_size: Optional[int],
        loss_reduction: str = "mean",
        generator=None,
        secure_mode: bool = False,
    ):
        """
        Args:
            optimizer: wrapped optimizer.
            noise_multiplier: noise multiplier
            max_grad_norm: max grad norm used for gradient clipping
            expected_batch_size: batch_size used for averaging gradients. When using
                Poisson sampling averaging denominator can't be inferred from the
                actual batch size. Required is ``loss_reduction="mean"``, ignored if
                ``loss_reduction="sum"``
            loss_reduction: Indicates if the loss reduction (for aggregating the gradients)
                is a sum or a mean operation. Can take values "sum" or "mean"
            generator: torch.Generator() object used as a source of randomness for
                the noise
            secure_mode: if ``True`` uses noise generation approach robust to floating
                point arithmetic attacks.
                See :meth:`~opacus.optimizers.optimizer._generate_noise` for details
        """
        if loss_reduction not in ("mean", "sum"):
            raise ValueError(f"Unexpected value for loss_reduction: {loss_reduction}")

        if loss_reduction == "mean" and expected_batch_size is None:
            raise ValueError(
                "You must provide expected batch size of the loss reduction is mean"
            )

        self.original_optimizer = optimizer
        self.noise_multiplier = noise_multiplier
        self.max_grad_norm = max_grad_norm
        self.loss_reduction = loss_reduction
        self.expected_batch_size = expected_batch_size
        self.step_hook = None
        self.generator = generator
        self.secure_mode = secure_mode

        self.param_groups = self.original_optimizer.param_groups
        self.defaults = self.original_optimizer.defaults
        self.state = self.original_optimizer.state
        self._step_skip_queue = []
        self._is_last_step_skipped = False

        for p in self.params:
            p.summed_grad = None

    def _get_flat_grad_sample(self, p: torch.Tensor):
        """
        Return parameter's per sample gradients as a single tensor.
        By default, per sample gradients (``p.grad_sample``) are stored as one tensor per
        batch basis. Therefore, ``p.grad_sample`` is a single tensor if holds results from
        only one batch, and a list of tensors if gradients are accumulated over multiple
        steps. This is done to provide visibility into which sample belongs to which batch,
        and how many batches have been processed.
        This method returns per sample gradients as a single concatenated tensor, regardless
        of how many batches have been accumulated
        Args:
            p: Parameter tensor. Must have ``grad_sample`` attribute
        Returns:
            ``p.grad_sample`` if it's a tensor already, or a single tensor computed by
            concatenating every tensor in ``p.grad_sample`` if it's a list
        Raises:
            ValueError
                If ``p`` is missing ``grad_sample`` attribute
        """

        if not hasattr(p, "grad_sample"):
            raise ValueError(
                "Per sample gradient not found. Are you using GradSampleModule?"
            )
        if p.grad_sample is None:
            raise ValueError(
                "Per sample gradient is not initialized. Not updated in backward pass?"
            )
        if isinstance(p.grad_sample, torch.Tensor):
            ret = p.grad_sample
        elif isinstance(p.grad_sample, list):
            ret = torch.cat(p.grad_sample, dim=0)
        else:
            raise ValueError(f"Unexpected grad_sample type: {type(p.grad_sample)}")

        return ret

    def signal_skip_step(self, do_skip=True):
        """
        Signals the optimizer to skip an optimization step and only perform clipping and
        per sample gradient accumulation.
        On every call of ``.step()`` optimizer will check the queue of skipped step
        signals. If non-empty and the latest flag is ``True``, optimizer will call
        ``self.clip_and_accumulate``, but won't proceed to adding noise and performing
        the actual optimization step.
        It also affects the behaviour of ``zero_grad()``. If the last step was skipped,
        optimizer will clear per sample gradients accumulated by
        ``self.clip_and_accumulate`` (``p.grad_sample``), but won't touch aggregated
        clipped gradients (``p.summed_grad``)
        Used by :class:`~opacus.utils.batch_memory_manager.BatchMemoryManager` to
        simulate large virtual batches with limited memory footprint.
        Args:
            do_skip: flag if next step should be skipped
        """
        self._step_skip_queue.append(do_skip)

    def _check_skip_next_step(self, pop_next=True):
        """
        Checks if next step should be skipped by the optimizer.
        This is for large Poisson batches that get split into smaller physical batches
        to fit on the device. Batches that do not correspond to the end of a Poisson
        batch or thus `skipped` as their gradient gets accumulated for one big step.
        """
        if self._step_skip_queue:
            if pop_next:
                return self._step_skip_queue.pop(0)
            else:
                return self._step_skip_queue[0]
        else:
            return False

    @property
    def params(self) -> List[nn.Parameter]:
        """
        Returns a flat list of ``nn.Parameter`` managed by the optimizer
        """
        return params(self)

    @property
    def grad_samples(self) -> List[torch.Tensor]:
        """
        Returns a flat list of per sample gradient tensors (one per parameter)
        """
        ret = []
        for p in self.params:
            ret.append(self._get_flat_grad_sample(p))
        return ret

    @property
    def accumulated_iterations(self) -> int:
        """
        Returns number of batches currently accumulated and not yet processed.
        In other words ``accumulated_iterations`` tracks the number of forward/backward
        passed done in between two optimizer steps. The value would typically be 1,
        but there are possible exceptions.
        Used by privacy accountants to calculate real sampling rate.
        """
        vals = []
        for p in self.params:
            if not hasattr(p, "grad_sample"):
                raise ValueError(
                    "Per sample gradient not found. Are you using GradSampleModule?"
                )
            if isinstance(p.grad_sample, torch.Tensor):
                vals.append(1)
            elif isinstance(p.grad_sample, list):
                vals.append(len(p.grad_sample))
            else:
                raise ValueError(f"Unexpected grad_sample type: {type(p.grad_sample)}")

        if len(set(vals)) > 1:
            raise ValueError(
                "Number of accumulated steps is inconsistent across parameters"
            )
        return vals[0]

    def attach_step_hook(self, fn):
        """
        Attaches a hook to be executed after gradient clipping/noising, but before the
        actual optimization step.
        Most commonly used for privacy accounting.
        Args:
            fn: hook function. Expected signature: ``foo(optim: DPOptimizer)``
        """

        self.step_hook = fn

    def clip_and_accumulate(self):
        """
        Performs gradient clipping.
        Stores clipped and aggregated gradients into `p.summed_grad```
        """

        trans_grad_samples = [g.transpose(0, 1) for g in self.grad_samples]
        per_param_norms = [
            g.reshape(len(g), -1).norm(2, dim=-1) for g in trans_grad_samples
        ]
        per_sample_norms = torch.stack(per_param_norms, dim=1).norm(2, dim=1)
        per_sample_clip_factor = (self.max_grad_norm / (per_sample_norms + 1e-6)).clamp(
            max=1.0
        )

        for p in self.params:
            _check_processed_flag(p.grad_sample)
            grad_sample = self._get_flat_grad_sample(p)
            grad = torch.sum(grad_sample * per_sample_clip_factor, dim=1)

            if p.summed_grad is not None:
                p.summed_grad += grad
            else:
                p.summed_grad = grad

            _mark_as_processed(p.grad_sample)

    def add_noise(self):
        """
        Adds noise to clipped gradients. Stores clipped and noised result in ``p.grad``
        """

        for p in self.params:
            _check_processed_flag(p.summed_grad)

            noise = _generate_noise(
                std=self.noise_multiplier * self.max_grad_norm,
                reference=p.summed_grad,
                generator=self.generator,
                secure_mode=self.secure_mode,
            )
            p.grad = (p.summed_grad + noise).view_as(p)

            _mark_as_processed(p.summed_grad)

    def scale_grad(self):
        """
        Applies given ``loss_reduction`` to ``p.grad``.
        Does nothing if ``loss_reduction="sum"``. Divides gradients by
        ``self.expected_batch_size`` if ``loss_reduction="mean"``
        """
        if self.loss_reduction == "mean":
            for p in self.params:
                p.grad /= self.expected_batch_size * self.accumulated_iterations

    def zero_grad(self, set_to_none: bool = False):
        """
        Clear gradients.
        Clears ``p.grad``, ``p.grad_sample`` and ``p.summed_grad`` for all of it's parameters
        Notes:
            ``set_to_none`` argument only affects ``p.grad``. ``p.grad_sample`` and
            ``p.summed_grad`` is never zeroed out and always set to None.
            Normal grads can do this, because their shape is always the same.
            Grad samples do not behave like this, as we accumulate gradients from different
            batches in a list
        Args:
            set_to_none: instead of setting to zero, set the grads to None. (only
            affects regular gradients. Per sample gradients are always set to None)
        """

        for p in self.params:
            p.grad_sample = None

            if not self._is_last_step_skipped:
                p.summed_grad = None

        self.original_optimizer.zero_grad(set_to_none)

    def pre_step(
        self, closure: Optional[Callable[[], float]] = None
    ) -> Optional[float]:
        """
        Perform actions specific to ``DPOptimizer`` before calling
        underlying  ``optimizer.step()``
        Args:
            closure: A closure that reevaluates the model and
                returns the loss. Optional for most optimizers.
        """
        self.clip_and_accumulate()
        if self._check_skip_next_step():
            self._is_last_step_skipped = True
            return False

        self.add_noise()
        self.scale_grad()

        if self.step_hook:
            self.step_hook(self)

        self._is_last_step_skipped = False
        return True

    def step(self, closure: Optional[Callable[[], float]] = None) -> Optional[float]:
        if closure is not None:
            with torch.enable_grad():
                closure()

        if self.pre_step():
            return self.original_optimizer.step()
        else:
            return None

    def __repr__(self):
        return self.original_optimizer.__repr__()

    def state_dict(self):
        return self.original_optimizer.state_dict()

    def load_state_dict(self, state_dict) -> None:
        self.original_optimizer.load_state_dict(state_dict)

In [75]:
@register_grad_sampler(ParallelOp)
def grad_sampler_parallel_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    return {}

@register_grad_sampler(MixedOp)
def grad_sampler_mixed_op(layer: MixedOp, activations: torch.Tensor, backprops: torch.Tensor):
    sftmx = torch.softmax(layer.alphas, 0)
    j_sftmx_vals = []
    for i in range(len(sftmx)):
        col = []
        for j in range(len(sftmx)):
            if i == j:
                deriv = sftmx[i] * (1 - sftmx[i])
            else:
                deriv = -sftmx[j] * sftmx[i]
            col.append(deriv)
        j_sftmx_vals.append(col)
    j_sftmx_trans = torch.tensor(j_sftmx_vals)
    
    #print(torch.all(activations[0] == 0))
    # d = c = number of operations, b = number of batches
    sftmx_grad = torch.einsum('dc,cb...->db...', j_sftmx_trans, activations) # we sum over columns since we have the transposed jacobian of softmax w.r.t. inputs
    final_grad = torch.einsum('db...,b...->db', sftmx_grad, backprops)
    #grad = torch.einsum('nbcwh,bcwh->nb', activations, backprops)
    ret = {
        layer.alphas: final_grad
    }
    return ret


criterion = nn.CrossEntropyLoss()
device = torch.device('cpu')
model_dp = Network(16, 10, 4, criterion, device, in_channels=1) # Cell(4, 3, 16, 36, 48, False, False)
optim_arch = torch.optim.SGD(get_params(model_dp, 'arch'), 0.01)
optim_model = torch.optim.SGD(get_params(model_dp, 'model'), 0.01)
pe = PrivacyEngine()
train_loader_c = deepcopy(train_loader)
model_dp_, _, train_loader_ = pe.make_private(module=model_dp, optimizer=optim_arch, data_loader=train_loader_c, noise_multiplier=1., max_grad_norm=1.)
optim_ = DPOptimizer(optim_arch, noise_multiplier=1., max_grad_norm=1., expected_batch_size=64)
x_first, y_first = next(iter(train_loader_))

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/opacus/privacy_engine.py:130: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


In [76]:
arch_params_before = [deepcopy(p) for n, p in model_dp_.named_parameters() if 'alpha' in n]
optim_.zero_grad()
y_hat = model_dp_(x_first)
l = criterion(y_hat, y_first)
l.backward()
arch_params_update = [deepcopy(p) for n, p in model_dp_.named_parameters() if 'alpha' in n]
optim_.step()
arch_params_after = [deepcopy(p) for n, p in model_dp_.named_parameters() if 'alpha' in n]

/home/jonas/anaconda3/envs/automl/lib/python3.9/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [77]:
for p1, p2 in zip(arch_params_after, arch_params_before):
    print(torch.all(p1.data == p2.data))

tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)
tensor(False)


In [78]:
arch_params_before[0]

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [83]:
arch_params_after[0]

Parameter containing:
tensor([ 6.0219e-05,  1.1829e-05,  1.3796e-04, -4.2296e-05,  1.5666e-04,
         2.0267e-05,  2.0030e-05, -8.0712e-05], requires_grad=True)